This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [1]:
import platform
import os
import pandas
import glob

In [2]:
# Copied from https://codereview.stackexchange.com/a/229889
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog

class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        with out:
            try:
                # Create Tk root
                root = Tk()
                # Hide the main window
                root.withdraw()
                # Raise the root to the top of all windows.
                root.call('wm', 'attributes', '.', '-topmost', True)
                # List of selected fileswill be set to b.value
                b.files = filedialog.askopenfilename(multiple=True)

                b.description = "Files Selected"
                b.icon = "check-square-o"
                b.style.button_color = "lightgreen"
            except:
                pass
out = widgets.Output()
raw = SelectFilesButton()
widgets.VBox([raw, out])

We can also select a folder and go through *each* subfolder there...

In [86]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
    BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
else:
    BasePath = os.path.join('R:\\')
# Jean
Root = os.path.join(BasePath, 'Rabbit-Grenoble')
# ZMK
Root = os.path.join(BasePath, 'ZahnmedizinischeKlinik', 'ToothBattallion')
Root = os.path.join(BasePath, 'ZMK', 'ToothBattallion')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/1272/ZMK/ToothBattallion


In [87]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [89]:
# Look only for folders: https://stackoverflow.com/a/38216530
# Jean
# Data['Folder'] = sorted(glob.glob(os.path.join(Root, '**', 'Ra*' + os.path.sep)))
# ZMK
Data['Folder'] = sorted(glob.glob(os.path.join(Root, '*' + os.path.sep)))
Data['Scan'] = [os.path.basename(os.path.split(f)[0]) for f in Data['Folder']]

In [90]:
# Grab a bunch of log files from a folder
# We could do it in a list comprehension, but then it fails if we're still scanning a tooth
# Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]
for c, row in Data.iterrows():
    try:
        # Jean
#         Data.at[c,'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], 'proj', '*.log')))[0]
        # ZMK
        Data.at[c,'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], '*.log')))[0]
    except IndexError:
        print('No logfile found in %s, removing the folder' % row.Folder)
        Data.at[c,'LogFile'] = 'scanning'
Data = Data[Data['LogFile'] != 'scanning']
Data.reset_index(drop=True, inplace=True)
print('We have %s tooth folders to work with' % (len(Data)))

We have 104 tooth folders to work with


In [614]:
# Choose the file selected manually
log = raw.files[0]

In [91]:
# Choose a file from the Dataframe with *all* log files
log = Data.LogFile[66]

In [92]:
print('We are looking at the values in %s' % log)

We are looking at the values in /home/habi/1272/ZMK/ToothBattallion/65/Tooth065.log


In [93]:
# Load log file manually (probably needed on Windows)
# log = 'f:\Verdiana Lung\Experiment01\Mouse04\proj\Mouse04.log'

In [94]:
# log = '/home/habi/research-storage-uct/Archiv_Tape/Rabbit-Grenoble/Rabbit-2-Grenoble2015/Rabbit-2-Overview-Grenoble2015-26-5um/proj/Rabbit-2-Overview-Grenoble2015-26-5um_.log'

In [95]:
# log = '/home/habi/1272/ZMK/ToothBattallion/99/Tooth099.log'

In [96]:
def fulllog(logfile):
    with open(logfile, 'r') as f:
        for line in f:
            print(line.strip())
    return()

In [97]:
# fulllog(log)

In [98]:
import re
def scanner(logfile, verbose=False):
    hardwareversion = []
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                machine = re.split('Sky.can', line)[1].strip()
            if 'Hardware' in line:
                if verbose:
                    print(line)
                hardwareversion = line.split('=')[1].strip()
    if hardwareversion:
        return('SkyScan %s (Version %s)' % (machine, hardwareversion))
    else:
        return('SkyScan ' + machine)    

In [99]:
# scanner(log, verbose=True)

In [100]:
def scansoftware(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Software' in line:
                if verbose:
                    print(line)
                version = line.split('=')[1].strip()
    return(version)

In [101]:
# fulllog(log)

In [102]:
# scansoftware(log, verbose=True)

In [103]:
def source(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source T' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                source = line.split('=')[1].strip()
    return(source)

In [104]:
# source(log, verbose=True)

In [105]:
def camera(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Camera T' in line or 'Camera=' in line:
                if verbose:
                    print(line)
                cam = line.split('=')[1].strip().strip(' camera')
    return(cam)

In [106]:
# camera(log, verbose=True)

In [107]:
def voltage(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Voltage' in line:
                if verbose:
                    print(line)
                V = float(line.split('=')[1])
    return(V)

In [108]:
def current(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Current' in line:
                if verbose:
                    print(line)
                A = float(line.split('=')[1])
    return(A)

In [109]:
# current(log, verbose=True)

In [110]:
def whichfilter(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                if verbose:
                    print(line)
                fltr = line.split('=')[1].strip().replace('  ', ' ')
                if fltr=='No Filter':
                    fltr=False
    return(fltr)

In [111]:
# whichfilter(log, verbose=True)

In [112]:
def numproj(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'f Files' in line:
                if verbose:
                    print(line)
                numproj = int(line.split('=')[1])
    return(numproj)

In [113]:
# numproj(log, verbose=True)

In [114]:
def stacks(logfile, verbose=False):
    with open(logfile, 'r') as f:
        # If only one stack, then there's nothing in the log file
        numstacks = 0
        for line in f:
            if 'b-scan' in line:
                if verbose:
                    print(line)
                # The 'Sub-scan scan length' is listed in the log file
                # We simply select the last one, and add 1, since Bruker also starts to count at zero
                numstacks = int(line.split('[')[1].split(']')[0])
    return(numstacks + 1)

In [115]:
# stacks(log, verbose=True)

In [116]:
def camerasize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Columns' in line:
                if verbose:
                    print(line)
                columns = int(line.split('=')[1])
            if 'Rows' in line:
                if verbose:
                    print(line)
                rows = int(line.split('=')[1])
    return(columns, rows)

In [117]:
def rotationstep(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                if verbose:
                    print(line)
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [118]:
def exposure(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                if verbose:
                    print(line)
                exp = int(line.split('=')[1])
    return(exp)

In [119]:
def averaging(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Avera' in line:
                if verbose:
                    print(line)
                details = line.split('=')[1]
                if 'ON' in details:
                    # https://stackoverflow.com/a/4894156/323100
                    avg = int(details[details.find("(")+1:details.find(")")])
                else:
                    avg=False
    return(avg)

In [238]:
import datetime
def duration(logfile, verbose=False):
    '''Returns scantime in *seconds*'''
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                if verbose:
                    print(line)
                duration = line.split('=')[1].strip()
    # Sometimes it's '00:24:26', sometimes '0h:52m:53s' :-/
    if 'h' in duration:
        scantime = datetime.datetime.strptime(duration, '%Hh:%Mm:%Ss')
    else:
        scantime = datetime.datetime.strptime(duration, '%H:%M:%S')
    return((scantime-datetime.datetime(1900,1,1)).total_seconds())

In [265]:
# log = '/home/habi/1272/Chondrules Space Yogita/NWA-200813/rec/NWA_rec.log'

In [299]:
def timeformat(tdelta, fmt):
    # From https://stackoverflow.com/a/8907269/323100
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

In [300]:
# log = Data.LogFile[66]

In [301]:
print('Each stack took', end=' ')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
print('to scan')

Each stack took 13 hours and 36 minutes to scan


In [122]:
def pixelsize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                if verbose:
                    print(line)
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [123]:
def version(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Reconstruction Program' in line:
                if verbose:
                    print(line)
                Program = line.split('=')[1].strip()
            if 'Program Version' in line:
                if verbose:
                    print(line)
                Version = line.split('sion:')[1].strip()
    return(Program, Version)

In [124]:
def ringremoval(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring' in line:
                if verbose:
                    print(line)
                ring = int(line.split('=')[1].strip())
    return(ring)

In [125]:
# ringremoval(log, verbose=True)

In [126]:
def beamhardening(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'ardeni' in line:
                if verbose:
                    print(line)
                bh = int(line.split('=')[1].strip())
    return(bh)

In [127]:
# beamhardening(log, verbose=True)

----

My microct blurb from http://simp.ly/publish/NBhZhH

In [128]:
print('Based on %s' % log)

Based on /home/habi/1272/ZMK/ToothBattallion/65/Tooth065.log


In [361]:
print('After $PREPARATION, the $NUMSAMPLES samples were imaged on a Bruker',
      scanner(log),
      'high-resolution microtomography machine (Control software version',
      scansoftware(log) +
      ', Bruker microCT, Kontich, Belgium).')

After $PREPARATION, the $NUMSAMPLES samples were imaged on a Bruker SkyScan 1272 high-resolution microtomography machine (Control software version 1.1.19, Bruker microCT, Kontich, Belgium).


In [362]:
print('The machine is equipped with a',
      source(log, verbose=False),
      'X-ray source and a',
      camera(log) +
      ' camera.')

The machine is equipped with a HAMAMATSU_L11871_20 X-ray source and a XIMEA xiRAY16 camera.


In [363]:
print('The X-ray source was set to a tube voltage of', 
      voltage(log),
      'kV and a tube current of',
      current(log),
      'µA, the x-ray spectrum was', end=' ')
if whichfilter(log):
    print('filtered by', whichfilter(log), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

The X-ray source was set to a tube voltage of 80.0 kV and a tube current of 125.0 µA, the x-ray spectrum was filtered by Al 1mm prior to incidence onto the sample.


In [365]:
print('For each sample, we recorded a set of', end=' ')
if stacks(log) > 1:
    print(stacks(log), 'stacked scans overlapping the sample height, each stack was recorded stack', end=' ')
print(numproj(log),
      'projections of',
      camerasize(log)[0],
      'x',
      camerasize(log)[1],
      'pixels at every',
      str(rotationstep(log)) + '° over a 180° sample rotation.')

For each sample, we recorded a set of 5 stacked scans overlapping the sample height, each stack was recorded stack 482 projections of 1632 x 1092 pixels at every 0.4° over a 180° sample rotation.


In [366]:
print('Every single projection was exposed for',
      exposure(log),
      'ms,',
      averaging(log),
      'projections were averaged to one to greatly reduce image noise.')

Every single projection was exposed for 950 ms, 3 projections were averaged to one to greatly reduce image noise.


In [371]:
print('This resulted in a scan time of approximately ', end='')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
if not stacks(log) == 1:
    print('per stack and about',
          timeformat(stacks(log) * datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample')
if stacks(log) == 1:
    print('.')
else:
    print('(with', stacks(log), 'stacks).')

This resulted in a scan time of approximately 39 minutes per stack and about 2 hours and 38 minutes per sample
(with 5 stacks).


In [372]:
print('In total, we scanned', Data.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      (Data.Duration.mean()-datetime.datetime(1900,1,1)).total_seconds()//60,
      'minutes.')
print('In total, we thus scanned for about', 
      timeformat(Data.Stacks.sum() *
                 datetime.timedelta(minutes=int((Data.Duration.mean() -
                                                 datetime.datetime(1900,1,1)).total_seconds()//60)),
                 '{days} days, {hours} hours and {minutes} minutes.'))
print('At the MIC rate, this would have cost', ((24 * 13) + 9) * 75, 'CHF!')

In total, we scanned 521 stacks.
Each stack took approximately 37.0 minutes.
In total, we thus scanned for about 13 days, 9 hours and 17 minutes.
At the MIC rate, this would have cost 24075 CHF!


In [376]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      version(log)[0],
      '(Version',
      version(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')
print('The whole process resulted in datasets with an isometric voxel size of',
      round(pixelsize(log),1),
      'µm.')    

The projection images were then subsequently reconstructed into a 3D stack of images with NRecon (Version 1.7.4.6, Bruker microCT, Kontich Belgium) using a ring artifact correction of 14.
The whole process resulted in datasets with an isometric voxel size of 10.0 µm.


In [138]:
# fulllog(log)

----

If we have several, then generate a dataframe...

In [377]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [scansoftware(log) for log in Data['LogFile']]

In [378]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]

In [379]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [380]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [381]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['CamSize'] = [camerasize(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]

In [382]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]

In [383]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [384]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [385]:
Data['Version'] = [version(log) for log in Data['LogFile']]

In [386]:
Data.to_excel('ZMK.xls')

In [387]:
Data.head()

,Folder,Scan,LogFile,Scanner,Software,Voxelsize,Source,Camera,Voltage,Current,...,Stacks,NumProj,CamSize,RotationStep,RingRemoval,Beamhardening,Exposure,Averaging,Duration,Version
0,/home/habi/1272/ZMK/ToothBattallion/1/,1,/home/habi/1272/ZMK/ToothBattallion/1/Tooth001...,SkyScan 1272,1.1.19,8.559493,HAMAMATSU_L11871_20,XIMEA xiRAY16,80.0,125.0,...,4,319,"(1632, 1092)",0.6,14,0,950,3,1589.0,"(NRecon, 1.7.4.6)"
1,/home/habi/1272/ZMK/ToothBattallion/10/,10,/home/habi/1272/ZMK/ToothBattallion/10/Tooth01...,SkyScan 1272,1.1.19,9.999986,HAMAMATSU_L11871_20,XIMEA xiRAY16,80.0,125.0,...,5,482,"(1632, 1092)",0.4,14,0,950,3,2432.0,"(NRecon, 1.7.4.6)"
2,/home/habi/1272/ZMK/ToothBattallion/100/,100,/home/habi/1272/ZMK/ToothBattallion/100/Tooth1...,SkyScan 1272,1.1.19,9.999986,HAMAMATSU_L11871_20,XIMEA xiRAY16,80.0,125.0,...,5,482,"(1632, 1092)",0.4,14,0,950,3,2385.0,"(NRecon, 1.7.4.6)"
3,/home/habi/1272/ZMK/ToothBattallion/101/,101,/home/habi/1272/ZMK/ToothBattallion/101/Tooth1...,SkyScan 1272,1.1.19,9.999986,HAMAMATSU_L11871_20,XIMEA xiRAY16,80.0,125.0,...,5,482,"(1632, 1092)",0.4,14,0,950,3,2385.0,"(NRecon, 1.7.4.6)"
4,/home/habi/1272/ZMK/ToothBattallion/102/,102,/home/habi/1272/ZMK/ToothBattallion/102/Tooth1...,SkyScan 1272,1.1.19,9.999986,HAMAMATSU_L11871_20,XIMEA xiRAY16,80.0,125.0,...,5,482,"(1632, 1092)",0.4,14,0,950,3,2390.0,"(NRecon, 1.7.4.6)"


In [388]:
for i in Data:
    print(10 * '-', i, 20 * '-')
    print(Data[i].unique())

---------- Folder --------------------
['/home/habi/1272/ZMK/ToothBattallion/1/'
 '/home/habi/1272/ZMK/ToothBattallion/10/'
 '/home/habi/1272/ZMK/ToothBattallion/100/'
 '/home/habi/1272/ZMK/ToothBattallion/101/'
 '/home/habi/1272/ZMK/ToothBattallion/102/'
 '/home/habi/1272/ZMK/ToothBattallion/103/'
 '/home/habi/1272/ZMK/ToothBattallion/104/'
 '/home/habi/1272/ZMK/ToothBattallion/11/'
 '/home/habi/1272/ZMK/ToothBattallion/12/'
 '/home/habi/1272/ZMK/ToothBattallion/13/'
 '/home/habi/1272/ZMK/ToothBattallion/14/'
 '/home/habi/1272/ZMK/ToothBattallion/15/'
 '/home/habi/1272/ZMK/ToothBattallion/16/'
 '/home/habi/1272/ZMK/ToothBattallion/17/'
 '/home/habi/1272/ZMK/ToothBattallion/18/'
 '/home/habi/1272/ZMK/ToothBattallion/19/'
 '/home/habi/1272/ZMK/ToothBattallion/2/'
 '/home/habi/1272/ZMK/ToothBattallion/20/'
 '/home/habi/1272/ZMK/ToothBattallion/21/'
 '/home/habi/1272/ZMK/ToothBattallion/22/'
 '/home/habi/1272/ZMK/ToothBattallion/23/'
 '/home/habi/1272/ZMK/ToothBattallion/24/'
 '/home/habi

In [389]:
fulllog(log)

[System]
Scanner=SkyScan1272
Instrument S/N=15G09089-B
Software Version=1.1.19
Home Directory=C:\SkyScan1272\SkyScan1272
Source Type=HAMAMATSU_L11871_20
Camera Type=XIMEA xiRAY16
Camera Pixel Size (um)=7.4
Camera X/Y Ratio=1.0023
[User]
User Name=haberthu
Computer Name=ANAMIC02
[Acquisition]
Data Directory=D:\Results\ZMK\ToothBattallion\\65
Filename Prefix=Tooth065~00
Filename Index Length=8
Number Of Files=  482
Number Of Rows= 1092
Number Of Columns= 1632
Partial Width=OFF
Image crop origin X=0
Image crop origin Y=0
Camera binning=3x3
Image Rotation=0.13600
Optical Axis (line)=  580
Camera to Source (mm)=174.07267
Object to Source (mm)=78.41100
Source Voltage (kV)=  80
Source Current (uA)= 125
Image Pixel Size (um)=9.999986
Scaled Image Pixel Size (um)=9.999986
Image Format=TIFF
Depth (bits)=16
Reference Intensity=57000
Camera position=close
Exposure (ms)=950
Rotation Step (deg)=0.400
Use 360 Rotation=NO
Scanning position=36.000 mm
Frame Averaging=ON (3)
Random Movement=OFF (30)
Flat

()